### Testando um algoritmo de inteligência artificial para a otimização de parâmetros
Ideia: 
 * Escrever todos os parâmetros de cada filtro utilizado no processo como uma lista
 * Essa lista representa um estado
 * O estado é então testado para o conjunto de teste (base de dados contento imagens e contagens)
 * É determinado o erro (diferença entre resultado e número de grãos real) médio para cada estado
 * O algoritmo parte de um estado inicial, gera todos os seus vizinhos, e determina o erro médio para cada vizinho
 * Então, escolhe o vizinho de menor erro
 * Se este vizinho tiver erro menor que o estado inicial, ele é passado para a próxima iteraçao
 * Senão, ele substitui o atual com probabilidade 𝑒^(Δ𝐸/T), sendo Δ𝐸 a diferença entre a função-objetivo desse estado com o estado atual e uma 𝑇 a temperatura que é reduzida a cada iteração.
 * O comportamento inicial do algoritmo é de ser mais permissivo quanto a piora do estado mas, com o passar das iterações, ele tende a aceitar apenas estados que apresentam melhoras. 
 * Se a temperatura for diminuída devagar o suficiente, esse algoritmo encontra o ótimo global com probabilidade se aproximando de 1

Fonte do algoritmo Simulated Annealing:

http://professor.ufabc.edu.br/~denis.fantinato/teaching/IA.html
http://professor.ufabc.edu.br/~denis.fantinato/teaching/Aula09.pdf
http://professor.ufabc.edu.br/~denis.fantinato/teaching/SimulatedAnnealing.py


In [1]:
#Bibliotecas
import cv2
import numpy as np
import matplotlib.pyplot as plt

#Função de utilidade que gera um kernel redondo
def getCircle(n):
    '''kernel has size NxN'''
    # xx and yy are 200x200 tables containing the x and y coordinates as values
    # mgrid is a mesh creation helper
    xx, yy = np.mgrid[:n,:n]
    # circles contains the squared distance to the (100, 100) point
    # we are just using the circle equation learnt at school
    circle = (xx - np.floor(n/2)) ** 2 + (yy - np.floor(n/2)) ** 2
    circle = circle<=np.max(circle)*.5
    circle = np.uint8(circle)
    return circle

#Função de processamento principal
def processing(img, parameters):
    '''
    parameters:
    0 thresholdType = cv2.ADAPTIVE_THRESH_MEAN_C or cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    1 blockSize = 99-299
    2 constant = 0-20
    3 kernelSize = [3-7]
    4 openingIt = 0-2
    5 erosionIt = 0-5
    6 contourMethod = cv2.CHAIN_APPROX_SIMPLE or cv2.CHAIN_APPROX_TC89_L1 or cv2.CHAIN_APPROX_TC89_KCOS 
    7 minArea = 15-50
    
    return img_contours, img_borders, img_colored, resultado, erro
    '''
    #Default parameters
    if parameters == None:
        parameters = [0,199,3,3,0,3,0,20]
    
    #Transformação do vetor de parâmetros em variáveis com nomes informativos
    if parameters[0] == 0: thresholdType = cv2.ADAPTIVE_THRESH_MEAN_C
    if parameters[0] == 1: thresholdType = cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    blockSize =  parameters[1]
    constant =   parameters[2]
    kernelSize = parameters[3]
    openingIt =  parameters[4]
    erosionIt =  parameters[5]
    if parameters[6] == 0: contourMethod = cv2.CHAIN_APPROX_SIMPLE
    if parameters[6] == 1: contourMethod = cv2.CHAIN_APPROX_TC89_L1
    if parameters[6] == 2: contourMethod = cv2.CHAIN_APPROX_TC89_KCOS
    minArea =    parameters[7]
    
    #Conversão de uma imagem para outro sistema de cores
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #Adaptivo
    img_thresh = cv2.adaptiveThreshold(img_gray, 255, thresholdType, cv2.THRESH_BINARY, blockSize, constant) 

    #Fechamento   
    kernel = getCircle(kernelSize)
    img_open = cv2.morphologyEx(img_thresh,cv2.MORPH_CLOSE,kernel, iterations = openingIt)
    img_open = cv2.erode(img_open, kernel, iterations=erosionIt)

    #Desenhando borda na imagem
    y,x = img_open.shape
    color = 0
    img_open[:,   0] = 0; img_open[:, x-1] = 0; img_open[0,   :] = 0; img_open[y-1, :] = 0

    #Gerando Lista de Contornos
    cv2MajorVersion = cv2.__version__.split(".")[0]
    if int(cv2MajorVersion) >= 4:
        contours, _= cv2.findContours(img_open,cv2.RETR_EXTERNAL, contourMethod)
    else:
        _, contours, _ = cv2.findContours(img_open,cv2.RETR_EXTERNAL, contourMethod)

    #Ordenando Lista de Contornos de acordo com a área
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    #Selecionando apenas contornos cuja área é maior que algum valor
    contours = [c for c in contours if cv2.contourArea(c)>minArea]

    #Desenhando Contornos na imagem original
    verde = (0,255,0)
    img_contours = cv2.drawContours(img.copy(), contours, -1, verde, 3)

    #Separando grãos das bordas
    faixa = 3
    n_borda = 0
    img_borders = np.int32(np.ones(img.shape))
    red = [0,255,0]
    blue = [255,0,0]
    for c in contours:
        (x_ini,y_ini,w,h) = cv2.boundingRect(c)
        x_end = x_ini+w; y_end = y_ini+h
        y_img, x_img = img_thresh.shape

        if 0<x_ini<faixa or 0<y_ini<faixa or x_img-faixa<x_end<x_img or y_img-faixa<y_end<y_img:
            n_borda +=1
            random_red = [np.random.randint(20, 235) for i in range(3)]
            random_red[0] = 255
            img_borders = cv2.fillPoly(img_borders, [c], random_red)
        else:
            random_blue = [np.random.randint(20, 235) for i in range(3)]
            random_blue[1] = 255
            img_borders = cv2.fillPoly(img_borders, [c], random_blue)

    #Preenchendo contornos
    img_colored = np.int32(np.ones(img.shape))
    img_out = img.copy()
    for c in contours:
        random_color = [np.random.randint(20, 235) for i in range(3)]
        img_colored = cv2.fillPoly(img_colored, [c], random_color)
        img_out = cv2.drawContours(img_out, [c], -1, random_color, 3)
    
    #resultados
    resultado = len(contours)-round(n_borda/2)
    
    return img_contours, img_borders, img_colored, img_out, resultado

In [2]:
#Construção de uma base de dados primitiva
img1 = plt.imread('data/Aço224.jpg')
img2 = plt.imread('data/Alu55.jpg')
img3 = plt.imread('data/Alu205.jpg')
images = [img1,img2,img3]
counts = [224,55,205]

In [9]:
#Definindo função de custo cujo resultado deverá ser minimizado pelo algoritmo
#   Essa função recebe um estado, processa na base de dados, 
#   calcula o erro do estado e retorna o erro médio
def costFunction(parameters, show=False):
    lista_erro = []
    for img, realN in zip(images,counts):
        
        _,_,_,_,resultado= processing(img, parameters)
        
        erro = abs(realN-resultado)
        
        lista_erro.append(erro)
    erro_medio = sum(lista_erro)/len(images)
    if show: print(lista_erro, erro_medio)
    return erro_medio

costFunction(None,show=True)

[16, 8, 251] 91.66666666666667


91.66666666666667

In [10]:
#0 thresholdType = cv2.ADAPTIVE_THRESH_MEAN_C or cv2.ADAPTIVE_THRESH_GAUSSIAN_C
#1 blockSize = 99-299
#2 constant = 0-20
#3 kernelSize = [3-7]
#4 openingIt = 0-2
#5 erosionIt = 0-5
#6 contourMethod = cv2.CHAIN_APPROX_SIMPLE or cv2.CHAIN_APPROX_TC89_L1 or cv2.CHAIN_APPROX_TC89_KCOS 
#7 minArea = 15-50
#s0 = list(np.random.randint(1,9, size=8))

#Função utilidade que testa se um estado está dentro das restrições de cada variável
def checkRange(sol):
    if not  0 <= sol[0] <=  1: return False
    if not(99 <= sol[1] <= 299 and sol[1]%2!=0): return False
    if not  0 <= sol[2] <= 20: return False
    if not( 3 <= sol[3] <=  7 and sol[3]%2!=0): return False
    if not  0 <= sol[4] <=  2: return False
    if not  0 <= sol[5] <=  5: return False
    if not  0 <= sol[6] <=  2: return False
    if not 15 <= sol[7] <= 50: return False
    return True

#Função que gera todos os vizinhos de um estado
#   Na minha definição, um vizinho é um estado com uma adição ou subtração em um dos parâmetros,
#   desde que essa operação gere um estado válido de acordo com a função acima
def vizinhanca(sol):
    vizinhos = []
    for p in range(len(sol)):
        vizinho = sol.copy()
        if p==1:
            vizinho[p]+=6
            if checkRange(vizinho): vizinhos.append(vizinho.copy())
            vizinho[p]-=12
            if checkRange(vizinho): vizinhos.append(vizinho.copy())
        elif p==3: 
            vizinho[p]+=2
            if checkRange(vizinho): vizinhos.append(vizinho.copy())
            vizinho[p]-=4
            if checkRange(vizinho): vizinhos.append(vizinho.copy())
        else:
            vizinho[p]+=1
            if checkRange(vizinho): vizinhos.append(vizinho.copy())
            vizinho[p]-=2
            if checkRange(vizinho): vizinhos.append(vizinho.copy())
    return vizinhos

In [11]:
#Função que gera estado aleatório dentro das restrições
import random
def randomState():
    p0 = np.random.randint(2)
    p1 = random.randrange(99, 299+1, 2)
    p2 = np.random.randint(21)
    p3 = random.randrange(3, 7+1, 2)
    p4 = np.random.randint(3)
    p5 = np.random.randint(6)
    p6 = np.random.randint(3)
    p7 = np.random.randint(15,51)
    return [p0,p1,p2,p3,p4,p5,p6,p7]
s0 = randomState()
print(s0)
checkRange(s0)

[1, 133, 11, 3, 1, 5, 2, 46]


True

In [12]:
#Teste de tempo
start_time = time.time()
print(start_time)
costFunction(None,show=True)
end_time = time.time()
print("runtime", end_time-start_time, "seconds")

1584848072.6307642
[16, 8, 251] 91.66666666666667
runtime 0.32362890243530273 seconds


In [13]:
#Função principal adaptada do site do professor Denis da disciplina Inteligência Artificial
#   Na função original, apenas um dos vizinhos, escolhido aleatóriamente,
#   era avaliado pela função de custo. Nesta versão, todos são avaliados e o melhor é escolhido.
#   Esta função também calcula o tempo gasto em sua execução.
import time
from datetime import datetime

def SimulatedAnnealing(s):
    print("current Time", datetime.now().strftime("%H:%M:%S"))
    start_time = time.time()
    T = 10000
    epslon = 1e-3
    factor = 0.95
    
    counter = 0
    t = T
    while t > epslon:
        t = t*factor
        counter +=1
    print("iterations", counter)
    print("estimated runtime", (counter*0.32*len(s)*2)/60, "minutes")
    
    fs = costFunction(s)
    
    while T > epslon:
        vizinhos = vizinhanca(s0)
        #vi = np.random.choice(len(vizinhos))
        #v  = vizinhos[vi]
        fv_best = 1000
        for v in vizinhos:
            fv = costFunction(v)
            if fv<fv_best:
                fv_best = fv
                v_best  = v
        v = v_best
        fv = fv_best

        if fv < fs or np.random.random() <= np.exp( (fs-fv)/T ):
            s, fs = v, fv
        T = T*factor
    
    end_time = time.time()
    print("runtime", (end_time-start_time)/60, "minutes")
    return s

In [331]:
#Simulação final

#s0 = [0,199,3,3,0,3,0,20]
s0 = randomState()
s = SimulatedAnnealing(s0)       
print(s0, s, costFunction(s,show=True))

current Time 23:46:38
iterations 315
estimated runtime 26.88 minutes
runtime 19.839790697892507 minutes
[102, 46, 18] 55.333333333333336
[0, 281, 14, 5, 0, 1, 2, 40] [0, 281, 14, 7, 0, 1, 2, 40] 55.333333333333336


In [196]:
#Resultados preliminares

#[102, 46, 18] 55.333333333333336
#[0, 281, 14, 5, 0, 1, 2, 40] [0, 281, 14, 7, 0, 1, 2, 40] 55.333333333333336

#[11, 2, 24] 12.333333333333334
#[1, 191, 2, 5, 0, 2, 0, 41] [1, 191, 3, 5, 0, 2, 0, 41] 12.333333333333334

#[0, 201, 5, 7, 1, 3, 1, 18] [0, 201, 5, 7, 0, 3, 1, 18] 69.33333333333333

#[0, 199, 3, 3, 0, 3, 0,20] [0, 199, 3, 5, 0, 3, 0, 20] 26.666666666666668

In [19]:
#teste
s = [1, 191, 3, 5, 0, 2, 0, 41]
print(s, costFunction(s,show=True))
img_out,_,_,_,result = processing(images[2],s)
print(result)

[11, 2, 24] 12.333333333333334
[1, 191, 3, 5, 0, 2, 0, 41] 12.333333333333334
181
